In [1]:
import json
import urllib.parse
import urllib.request
import base64
from datetime import datetime
import pandas as pd

# Stałe
service_url = 'https://shapi.imgw.pl/observation/read-many'
user_login = 'alejur'            # <----------------------------------- zmiana loginu
user_password = 'qwer1234'       # <----------------------------------- zmiana hasła na swoje

auth_string = f'{user_login}:{user_password}'.encode('utf-8')
auth_base64 = base64.b64encode(auth_string).decode('utf-8')
headers = {
    'Authorization': f'Basic {auth_base64}',
    'Content-Type': 'application/json'
}

# Wczytanie listy stacji
stations_df = pd.read_csv('stacje_do_SPEI_92.csv', encoding='cp1250')   # <--------------plik ze stacjami (jesli znajduję się w jakiejś innej lokalizacji trzeba wpisać całą ścieżke)
stations_df["id"] = stations_df["id"].astype(str)

# Funkcja pobierająca dane z SH API
def fetch_data(station_ids, parameter):
    values = {
        'dre-names': ','.join(station_ids),
        'parameter-ids': parameter,
        'from': '20250501000000',   # <------------------------------- zmiana daty 
        'to':   '20250531235900',   # <------------------------------- zmiana daty   (musi byc w tej postaci)
        'time-specification': '10'
    }
    url_values = urllib.parse.urlencode(values)
    full_url = f'{service_url}?{url_values}'
    req = urllib.request.Request(full_url, headers=headers, method='GET')
    try:
        with urllib.request.urlopen(req) as response:
            content = response.read()
            return json.loads(content)
    except:
        return []

# Funkcja przetwarzająca dane do DataFrame
def process_records(records):
    data = []
    for record in records:
        try:
            station_id = record["dre-name"]
            date = datetime.strptime(str(record["date"]), '%Y%m%d%H%M%S')
            temp_value = float(record["value"])
            data.append([station_id, date, temp_value])
        except:
            continue
    if not data:
        return pd.DataFrame()
    df = pd.DataFrame(data, columns=["station_id", "timestamp", "temperature"])
    df["date"] = df["timestamp"].dt.date
    df["hour"] = df["timestamp"].dt.strftime('%H:%M')

    grouped = df.groupby(["station_id", "date"])["temperature"].agg(["min", "max", "mean"]).reset_index()
    temps_at_hours = df[df["hour"].isin(["06:00", "18:00"])]
    pivoted = temps_at_hours.pivot_table(index=["station_id", "date"], 
                                         columns="hour", 
                                         values="temperature", 
                                         aggfunc="first").reset_index()
    pivoted.columns.name = None
    final_df = pd.merge(grouped, pivoted, on=["station_id", "date"], how="left")
    final_df["mean"] = final_df["mean"].round(2)
    
    return final_df

# === Krok 1: Pobierz dane z B00304A ===
station_ids = stations_df["id"].tolist()
records_main = fetch_data(station_ids, "B00302A")
df_main = process_records(records_main)

# Zidentyfikuj użyte stacje
if not df_main.empty and "station_id" in df_main.columns:
    used_ids = df_main["station_id"].astype(str).unique().tolist()
    remaining_ids = [sid for sid in station_ids if sid not in used_ids]
else:
    print("⚠️ df_main jest pusty lub brak kolumny 'station_id'. Pobieram dane dla wszystkich stacji.")
    remaining_ids = station_ids.copy()
    df_main = pd.DataFrame()  # upewnia się, że df_main istnieje dalej jako pusty

# === Krok 2: Pobierz dane z B00302A dla reszty ===
records_fallback = fetch_data(remaining_ids, "B00304A")
df_fallback = process_records(records_fallback)

# === Krok 3: Połącz dane i dołącz stacje ===
final_df = pd.concat([df_main, df_fallback], ignore_index=True)
final_df["station_id"] = final_df["station_id"].astype(str)
stations_df["id"] = stations_df["id"].astype(str)

merged_df = pd.merge(final_df, stations_df[["id", "Nazwa.stacji", "rank_code"]],
                     left_on="station_id", right_on="id", how="left")
merged_df.drop(columns=["station_id"], inplace=True)

merged_df.rename(columns={
"06:00": "godz_6",
"18:00": "godz_18"
}, inplace=True)

# === Zapis do CSV ===  <----------------- zmien nazwe pliku lub ściezke
merged_df.to_csv("B:/CHOK/BMPIS/CMPIS-K/2_wymiana_Danych/__SUSZA/BIULETYN/01_BIULETYN_OLA_RADEK/SPEI_dane/KOD_SCIAGANIE_DANYCH_PUBLICZNYCH/SPEI_ETO/SPEI_ETO/SCIAGANIE_NOWYCH_DANYCH_SH_API/dane_2025.04_05.csv", index=False, encoding='utf-8-sig')
print("✅ Dane zapisane")
merged_df

✅ Dane zapisane


,date,min,max,mean,godz_6,godz_18,id,Nazwa.stacji,rank_code
0,2025-05-01,7.8,22.4,15.36,13.4,17.2,249180010,PSZCZYNA,KLIMATYCZNA
1,2025-05-02,6.2,25.8,17.15,13.5,20.1,249180010,PSZCZYNA,KLIMATYCZNA
2,2025-05-03,13.5,27.2,19.17,18.1,15.9,249180010,PSZCZYNA,KLIMATYCZNA
3,2025-05-04,8.9,15.7,12.80,14.2,12.6,249180010,PSZCZYNA,KLIMATYCZNA
4,2025-05-05,6.7,12.2,8.90,8.8,8.0,249180010,PSZCZYNA,KLIMATYCZNA
...,...,...,...,...,...,...,...,...,...
2648,2025-05-27,9.1,20.7,15.39,13.0,18.7,350190566,KRAKÓW-BALICE,SYNOPTYCZNA
2649,2025-05-28,7.1,22.6,15.53,14.6,15.1,350190566,KRAKÓW-BALICE,SYNOPTYCZNA
2650,2025-05-29,8.4,17.2,14.00,13.4,15.1,350190566,KRAKÓW-BALICE,SYNOPTYCZNA
2651,2025-05-30,8.1,18.5,14.27,13.9,15.1,350190566,KRAKÓW-BALICE,SYNOPTYCZNA


In [9]:
merged_df.head(60)

,date,min,max,mean,godz_6,godz_18,id,Nazwa.stacji,rank_code
0,2025-05-01,7.80,22.40,15.36,13.40,17.20,249180010,PSZCZYNA,KLIMATYCZNA
1,2025-05-02,6.20,25.80,17.15,13.50,20.10,249180010,PSZCZYNA,KLIMATYCZNA
2,2025-05-03,13.50,27.20,19.17,18.10,15.90,249180010,PSZCZYNA,KLIMATYCZNA
3,2025-05-04,8.90,15.70,12.80,14.20,12.60,249180010,PSZCZYNA,KLIMATYCZNA
4,2025-05-05,6.70,12.20,8.90,8.80,8.00,249180010,PSZCZYNA,KLIMATYCZNA
5,2025-05-06,6.10,12.20,9.35,8.20,10.60,249180010,PSZCZYNA,KLIMATYCZNA
6,2025-05-07,4.20,14.70,10.27,9.20,11.10,249180010,PSZCZYNA,KLIMATYCZNA
7,2025-05-08,1.40,13.10,7.72,6.30,9.80,249180010,PSZCZYNA,KLIMATYCZNA
8,2025-05-09,0.30,14.10,7.59,7.10,9.40,249180010,PSZCZYNA,KLIMATYCZNA
9,2025-05-10,4.80,15.80,9.84,8.20,10.50,249180010,PSZCZYNA,KLIMATYCZNA


In [3]:
len(merged_df.id.unique().tolist())   # informacja ile jest unikalnych stacji

NameError: name 'merged_df' is not defined